In [ ]:
import warnings
warnings.filterwarnings('ignore')

# data imports
import pandas as pd
import numpy as np
from plotnine import *

# modeling imports
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, LogisticRegression # Linear Regression Model
from sklearn.preprocessing import StandardScaler, OneHotEncoder #Z-score variables
from sklearn.metrics import mean_squared_error, f1_score,r2_score, roc_auc_score,recall_score,mean_absolute_percentage_error, mean_absolute_error, accuracy_score,precision_score #model evaluation
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut

# pipeline imports
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer

%matplotlib inline


# Review



## Link Functions

In our Linear Regression lectures, we talked about adding non-linearity through **Feature Engineering**, but that's not the only way! We can also use **link functions** to add non-linearity.

Link functions are just algebra we do to the linear prediction ($\mathbf{X}\beta$) in order to get the predicted value we *actually* want (e.g a probability).

$$\underbrace{y = \mathbf{X}\beta}_\text{Linear Model}$$
$$\underbrace{y = g^{-1}(\mathbf{X}\beta)}_\text{Generalized Linear Model}$$

Oddly, we often specify our link function using it's *inverse*, hence the $g^{-1}()$ instead of $g()$. $g^{-1}()$ takes the linear prediction and transforms it into our desired predicted value. $g()$ takes our desired predicted value and transforms it back into our linear prediction.



### Logistic Regression
In logistic regression, our goal is to predict a *probability* that a data point is in group `1`. We talked about using:

- Linear Probability Models $g^{-1}: y = x$
- Odds Models $g^{-1}: y = e^x$
- Logistic Regression: $g^{-1}: y = \frac{e^x}{1 + e^x}$

Logistic Regression using the link function $g(x) = log{\frac{x}{1-x}}$ and inverse link $g^{-1}: y = \frac{e^x}{1 + e^x}$ gave us a great sigmoid shape that takes linear predictions ($y = \mathbf{X}\beta$) and turns them into predicted probabilities ($p = \frac{e^{\mathbf{X}\beta}}{1 + e^{\mathbf{X}\beta}}$).

## Maximum Likelihood Estimation

Just like with Linear Regression, we can use **Maximum Likelihood Estimation** to choose the parameters (intercept and coefficients) of the model. But we have a different likelihood.

In a linear regression, we assumed that our *errors* are *normally* distributed around the regression line. For logistic regression, we assume that our *errors* are *Bernoulli* distributed. The Bernoulli distribution is a discrete distribution (since our outcome is *discrete*, a.k.a categorical) that tells you the proability of being `0` or `1`.

### Bernoulli Likelihood
The formula for a Bernoulli distribution for a single data point $x$ is:

$$ f(y;p(x)) = p(x)^{y} * (1-p(x))^{1-y}$$

where $y$ is the group the data point belongs to (either `0` or `1`), and $p(x)$ is the predicted probability of that data point being a `1`.

For example, let's say we're looking at the probability that it's sunny tomorrow. The predicted probability, according to the weather channel is $p(x) = 0.8$. The likelihood of it being sunny ($k = 1$) is:

$$ f(1;0.8) = 0.8^1 * (1-0.8)^{1-1} = 0.8$$

The likelihood of it not being sunny ($k = 0$) is:
$$ f(0;0.8) = 0.8^0 * (1-0.8)^{1-0} = 0.2$$

### Likelihood Function
But we don't just have a SINGLE data point when fitting a logistic regression, we have MANY. So, we multiply the likelihood of each data point together to get the likelihood of the dataset:

$$\prod_{i = 1}^n p(x_i)^{y_i} * (1-p(x_i))^{1-y_i}$$

We want to choose parameters (e.g. $\beta_0$, or $\beta_1$) that *maximize* this likelihood function. And how to we maximize it? We take it's (partial) derivatives and set them equal to zero!

However, it turns out that its much easier to work with the *log* of this likelihood function, so we're often working with the *log* likelihood and taking it's derivatives (this will still find the optimal parameters for the model as the values that maximize the *log* likelihood will also maximize the likelihood):

$$\sum_{i = 1}^n y_i * log(p(x_i)) + (1-y_i) * log(1-p(x_i))$$

### Loss Function
Now it turns out, this log-loss is a really great **loss function** for logistic regression. Loss functions are metrics that

1. measure the performance of your model, and
2. have lower scores indicate better performing models

Log-Loss (also called **Binary Cross Entropy**) does just that! Thus we often use it as a loss function for Logistic Regression.

<img src="https://drive.google.com/uc?export=view&id=1jZBGKZCaDfP8g3mFpt0FArPc1r7H60Bs" alt="Q" width = "400"/>


##  Logistic Regression Example

Let's do a example from scratch using this [Breast Cancer](https://raw.githubusercontent.com/ywen2021/CPSC392/main/Data/BreastCancer.csv) data.

In [ ]:
# read in data
bc = pd.read_csv("https://raw.githubusercontent.com/ywen2021/CPSC392/main/Data/BreastCancer.csv")
### drop the unnamed last column
bc.drop(bc.columns[bc.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

bc.head()
bc['diagnosis'] = (bc['diagnosis'] == 'M').astype(int)
bc
# set X and y
predictors = bc.columns[bc.columns.str.endswith("mean")]

X = bc[predictors]
y = bc["diagnosis"]


# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)
# z-score

pre = make_column_transformer((StandardScaler(), predictors),
                              remainder = "passthrough")

# Create Empty Model
logreg = LogisticRegression()

pipe = Pipeline([("pre", pre), ("model", logreg)])



# fit
pipe.fit(X_train,y_train)

# predict
y_pred_train = pipe.predict(X_train)
y_pred_test = pipe.predict(X_test)

y_pred_train_prob = pipe.predict_proba(X_train)[:,1]
y_pred_test_prob = pipe.predict_proba(X_test)[:,1]


# assess
# assess
print("Train Acc       : ", accuracy_score(y_train, y_pred_train))
print("Train Prescision: ", precision_score(y_train, y_pred_train))
print("Train Recall    : ", recall_score(y_train, y_pred_train))
print("Train F1        : ", f1_score(y_train, y_pred_train))
print("Train ROC AUC   : ", roc_auc_score(y_train, y_pred_train_prob))


print("Test Acc        : ", accuracy_score(y_test, y_pred_test))
print("Test Prescision : ", precision_score(y_test, y_pred_test))
print("Test Recall     : ", recall_score(y_test, y_pred_test))
print("Test F1         : ", f1_score(y_test, y_pred_test))
print("Test ROC AUC    : ", roc_auc_score(y_test, y_pred_test_prob))

Train Acc       :  0.9428571428571428
Train Prescision:  0.9401197604790419
Train Recall    :  0.9075144508670521
Train F1        :  0.9235294117647059
Train ROC AUC   :  0.9884598040421433
Test Acc        :  0.9298245614035088
Test Prescision :  0.9428571428571428
Test Recall     :  0.8461538461538461
Test F1         :  0.8918918918918919
Test ROC AUC    :  0.9788034188034188


Quick note about Logistic Regression Coefficients:

1. Logistic Regression coefficients are by default in terms of *log odds* meaning that they tell you how much the *predicted log odds* of being in group 1 will change when the *predictor* increases by 1-unit.
2. In `sklearn`, Logistic Regression can actually handle outcomes with *more than 2* categories. For example we could predict what someone's major is using Logistic Regression. When we do that, `sklearn` actually builds *many* logistic regression models by default: one model per category (see [here](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression) for more rigorous detail.) That means there are *multiple* sets of coefficients, one for each model. Because of this, when there is only 2 categories (e.g. a *binary* outcome), we use `[0]` to indicate that we want to grab only one set of coeficients, because there is only one set in the *binary* case.

In [ ]:
coefficients = pd.DataFrame({"Coef": pipe.named_steps['model'].coef_[0], # grab array of coefficients
                            "Name": predictors})
intercept = pd.DataFrame({"Coef": pipe.named_steps['model'].intercept_[0], # grab intercept
                                   "Name": "intercept"},
                                   index = [coefficients.shape[0]]) # since this is only 1 row, assign row a row index

coefficents_all = pd.concat([coefficients,intercept])
coefficents_all

,Coef,Name
0,1.192640,radius_mean
1,1.424881,texture_mean
2,1.066551,perimeter_mean
3,1.317058,area_mean
4,0.852305,smoothness_mean
5,-0.297744,compactness_mean
6,0.997359,concavity_mean
7,1.432847,concave points_mean
8,0.682689,symmetry_mean
9,-0.361737,fractal_dimension_mean


# In Your Groups

## Practice Code
Now practice building Logistic Regression models in your groups. Using [this](https://raw.githubusercontent.com/ywen2021/CPSC392/main/Data/purchase.csv) dataset, build a Logistic Regression model that predicts whether or not customers signed up for a rewards program based on their age, income, and whether they had made a previous purchase. Make sure to z-score your continuous variables, and use an 80/20 Train-Test-Split.


In [ ]:
# read in data
df = pd.read_csv("https://raw.githubusercontent.com/ywen2021/CPSC392/main/Data/purchase.csv")

df.head()

# set X and y
predictors = ["age", "income_in_k", "previous_purchase"]

X = df[predictors]
y = df["signed_up"]

# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 392)

# Z-score and Create Empty Model


# fit


# predict


# assess


Now, grab the coefficients for the model:

In [ ]:
# coefs
coefficients = pd.DataFrame({"Coef": pipe.named_steps['logreg'].coef_[0], # grab array of coefficients
                            "Name": predictors})
intercept = pd.DataFrame({"Coef": pipe.named_steps['logreg'].intercept_[0], # grab intercept
                                   "Name": "intercept"},
                                   index = [coefficients.shape[0]]) # since this is only 1 row, assign row a row index

coefficents_all = pd.concat([coefficients,intercept])
coefficents_all

## The Problem with Logistic Regression Coefficients
When you're presenting your Logistic Regression Models to non-data people, you might want to be able to tell them which variables have the biggest impact on the predicted value. Typically, we might use coefficients for this because they give us a single number that summarizes the relationship between our *predictors* and our *predicted value*.

However, log odds are difficult to understand intuitively, especially if you're not a data person. Thus, we might want a different way to present our results. Luckily, if we *exponentiate* our **log odds** coefficients, we get **odds** coefficients. These are easier to understand, as most people understand intuitively what **odds** are.

Remember, for **odds** the important threshold value is $1$. So any **odds** coefficient $>1$ has a direct/positive relationship with the outcome and anything with an **odds** coefficient $< 1$ has an inverse/negative relationship with the outcome.

You can also use the **odds** coefs to give people an intuitive understanding of the relationship. If the odds coef is $2$ then increasing the predictor by 1-unit causes your predicted odds to *double*. Similarly, if the odds coef is $0.5$ then increasing the predictor by 1-unit causes your predicted odds to *halve*. If the odds coef is $1.25$ then increasing the predictor by 1-unit causes your predicted odds to increase by $25\%$.
